# Guardian Queries 

In [1]:
from data201 import db_connection, df_query
from pandas import DataFrame
conn = db_connection('sheql.ini')
cursor = conn.cursor()

### Dataframe visualization for testing purposes

In [2]:
def display_results(cursor):
    """
    If there are results in the cursor from a call to a
    stored procedure, display the results in a dataframe. 
    """
    columns = result.description
    
    if columns == None:
        return 
    
    else:
        column_names = [column_info[0] 
                        for column_info in columns]

        rows = cursor.fetchall()
    
        if len(rows) > 0:
            df = DataFrame(rows)
            df.columns = column_names
            
            display(df)
        
        else:
            return

# Display Guardian Name and Students 

In [3]:
cursor.execute('DROP PROCEDURE IF EXISTS get_guardian_info')

cursor.execute(
    """
    CREATE PROCEDURE get_guardian_info(
        IN gUserName VARCHAR(50)
    )
    BEGIN
        SELECT 
            CONCAT(g.first_name, ' ', g.last_name) AS guardian_name, 
            s.student_id, 
            s.first_name, 
            s.last_name
        FROM users u 
        JOIN guardian g USING (user_id) 
        JOIN guardian_student_relationship USING (guardian_id)
        JOIN student s USING (student_id)
        WHERE u.username = gUserName;
    END
    """
)

In [4]:
gUserName = 'gua_d.burch'

cursor.callproc('get_guardian_info', (gUserName,))

for result in cursor.stored_results():
    display_results(result)

,guardian_name,student_id,first_name,last_name
0,Donna Burch,384,Patrick,Burch


# Display Student Grades 

In [5]:
cursor.execute('DROP PROCEDURE IF EXISTS get_guardian_student_grades')

cursor.execute(
    """
    CREATE PROCEDURE get_guardian_student_grades(
        IN gUserName VARCHAR(50)
    )
    BEGIN
        SELECT 
            s.first_name AS student_first_name,
            s.last_name AS student_last_name,
            c.name AS course_name,
            ROUND(SUM(gd.score * gd.weight) / SUM(gd.weight), 2) AS weighted_grade,
            CASE
                WHEN ROUND(SUM(gd.score * gd.weight) / SUM(gd.weight), 2) >= 90 THEN 'A'
                WHEN ROUND(SUM(gd.score * gd.weight) / SUM(gd.weight), 2) >= 80 THEN 'B'
                WHEN ROUND(SUM(gd.score * gd.weight) / SUM(gd.weight), 2) >= 70 THEN 'C'
                WHEN ROUND(SUM(gd.score * gd.weight) / SUM(gd.weight), 2) >= 60 THEN 'D'
                ELSE 'F'
            END AS letter_grade
        FROM users u
        JOIN guardian g ON u.user_id = g.user_id
        JOIN guardian_student_relationship gs ON g.guardian_id = gs.guardian_id
        JOIN student s ON gs.student_id = s.student_id
        JOIN grade_details gd ON s.student_id = gd.student_id
        JOIN course c ON gd.course_id = c.course_id
        WHERE u.username = gUserName
        GROUP BY s.student_id, c.course_id, s.first_name, s.last_name, c.name
        ORDER BY s.last_name, s.first_name, c.name;
    END
    """
)

In [6]:
gUserName = 'gua_d.burch'

cursor.callproc('get_guardian_student_grades', (gUserName,))

for result in cursor.stored_results():
    display_results(result)

,student_first_name,student_last_name,course_name,weighted_grade,letter_grade
0,Patrick,Burch,Art,60.00,D
1,Patrick,Burch,Computers,77.50,C
2,Patrick,Burch,English,88.55,B
3,Patrick,Burch,History,80.75,B
4,Patrick,Burch,Math,75.75,C
5,Patrick,Burch,Science,82.55,B


# Display Student Attendance 

In [7]:
cursor.execute('DROP PROCEDURE IF EXISTS get_guardian_student_attendance')

cursor.execute(
    """
    CREATE PROCEDURE get_guardian_student_attendance(
        IN gUserName VARCHAR(50)
    )
    BEGIN
        SELECT
            s.first_name AS student_first_name,
            s.last_name AS student_last_name,
            a.date,
            a.status,
            a.notes,
            t.first_name AS recorded_by_first_name,
            t.last_name AS recorded_by_last_name
    FROM users u
    JOIN guardian g ON u.user_id = g.user_id
    JOIN guardian_student_relationship gs ON g.guardian_id = gs.guardian_id
    JOIN student s ON gs.student_id = s.student_id
    JOIN attendance a ON s.student_id = a.student_id
    LEFT JOIN teacher t ON a.recorded_by = t.teacher_id
    WHERE u.username = gUserName 
    ORDER BY s.last_name, s.first_name, a.date DESC;
    END
    """
)

In [8]:
gUserName = 'gua_d.burch'

cursor.callproc('get_guardian_student_attendance', (gUserName,))

for result in cursor.stored_results():
    display_results(result)

,student_first_name,student_last_name,date,status,notes,recorded_by_first_name,recorded_by_last_name
0,Patrick,Burch,2025-05-15,present,None,Richard,Brown
1,Patrick,Burch,2025-05-14,present,None,Richard,Brown
2,Patrick,Burch,2025-05-13,present,None,Richard,Brown
3,Patrick,Burch,2025-05-12,present,None,Richard,Brown
4,Patrick,Burch,2025-05-09,present,None,Richard,Brown
5,Patrick,Burch,2025-05-08,present,None,Richard,Brown
6,Patrick,Burch,2025-05-07,present,None,Richard,Brown
7,Patrick,Burch,2025-05-06,present,None,Richard,Brown
8,Patrick,Burch,2025-05-05,present,None,Richard,Brown
9,Patrick,Burch,2025-05-02,present,None,Richard,Brown


# Display Dates of Student Attendance by Status

In [9]:
cursor.execute('DROP PROCEDURE IF EXISTS get_guardian_student_attendance_dates')

cursor.execute(
    """
    CREATE PROCEDURE get_guardian_student_attendance_dates(
        IN gUserName VARCHAR(50)
    )
    BEGIN
        SELECT
            a.date,
            a.status
    FROM users u
    JOIN guardian g ON u.user_id = g.user_id
    JOIN guardian_student_relationship gs ON g.guardian_id = gs.guardian_id
    JOIN student s ON gs.student_id = s.student_id
    JOIN attendance a ON s.student_id = a.student_id
    LEFT JOIN teacher t ON a.recorded_by = t.teacher_id
    WHERE u.username = gUserName 
    ORDER BY s.last_name, s.first_name, a.date DESC;
    END
    """
)

In [10]:
gUserName = 'gua_d.burch'

cursor.callproc('get_guardian_student_attendance_dates', (gUserName,))

for result in cursor.stored_results():
    display_results(result)

,date,status
0,2025-05-15,present
1,2025-05-14,present
2,2025-05-13,present
3,2025-05-12,present
4,2025-05-09,present
5,2025-05-08,present
6,2025-05-07,present
7,2025-05-06,present
8,2025-05-05,present
9,2025-05-02,present


# Display Student Attendance Snapshot

In [11]:
cursor.execute('DROP PROCEDURE IF EXISTS get_student_attendance_snapshot')

cursor.execute(
    """
    CREATE PROCEDURE get_student_attendance_snapshot(
        IN gUserName VARCHAR(50),
        IN sID INT
    )
    BEGIN
        SELECT
            s.first_name,
            s.last_name,
            SUM(a.status = 'Present') AS present_count,
            SUM(a.status = 'Absent') AS absent_count,
            SUM(a.status = 'Late') AS late_count,
            SUM(a.status = 'Excused') AS excused_count
        FROM users u
        JOIN guardian g ON u.user_id = g.user_id
        JOIN guardian_student_relationship gs ON g.guardian_id = gs.guardian_id
        JOIN student s ON gs.student_id = s.student_id
        JOIN attendance a ON s.student_id = a.student_id
        WHERE u.username = gUserName AND s.student_id = sID
        GROUP BY s.student_id;
    END
    """
)

In [12]:
gUserName = 'gua_d.burch'
sID = 384

cursor.callproc('get_student_attendance_snapshot', (gUserName, sID))

for result in cursor.stored_results():
    display_results(result)

,first_name,last_name,present_count,absent_count,late_count,excused_count
0,Patrick,Burch,39,0,2,0


# Display Student's Homeroom Teacher 

In [13]:
cursor.execute('DROP PROCEDURE IF EXISTS get_guardian_student_teacher')

cursor.execute(
    """
    CREATE PROCEDURE get_guardian_student_teacher(
        IN gUserName VARCHAR(50),
        IN sID INT 
    )
    BEGIN
        SELECT 
            t.first_name AS teacher_first_name,
            t.last_name AS teacher_last_name,
            u_teacher.email AS teacher_email
        FROM users u_guardian
        JOIN guardian g ON u_guardian.user_id = g.user_id
        JOIN guardian_student_relationship gs ON g.guardian_id = gs.guardian_id
        JOIN student s ON gs.student_id = s.student_id
        JOIN teacher t ON s.homeroom_teacher_id = t.teacher_id
        JOIN users u_teacher ON t.user_id = u_teacher.user_id
        WHERE u_guardian.username = gUserName
          AND s.student_id = sID; 

    END
    """
)

In [14]:
gUserName = 'gua_d.burch'
sID = 384

cursor.callproc('get_guardian_student_teacher', (gUserName, sID))

for result in cursor.stored_results():
    display_results(result)

,teacher_first_name,teacher_last_name,teacher_email
0,Richard,Brown,tea_rich.brown@jones.com


In [15]:
cursor.close()
conn.close()